## Calc.py

In [1]:
import numpy as np


def calc_intrisk(sloccur, t_eff, alpharisk, betarisk, timeweight):
    slevnt = np.sum(np.multiply(sloccur, np.tile(np.transpose(np.power(timeweight, t_eff)), (1, len(sloccur[1, :])))),
                    0, keepdims=True)  # make sure to load t_eff as 2D array in order for this to work properly
    tmevnt = np.sum(np.power(timeweight, t_eff))
    sl_risk = (slevnt + alpharisk) / (tmevnt + alpharisk + betarisk)
    return sl_risk, slevnt, tmevnt


def calc_neival(c_trans, p_sl, y_node, q_node, lccf, rtpref, dtonei, cutflag, totcpcty, totstock, edgechange):
    pay_noevent = np.zeros((c_trans.shape[1], 1))  # len in numpy defaults to 1st dimension and shape provides a tuple
    pay_event = np.zeros((c_trans.shape[1], 1))
    xpay_noevent = np.zeros((c_trans.shape[1], 1))
    ypay_noevent = np.zeros((c_trans.shape[1], 1))
    xpay_event = np.zeros((c_trans.shape[1], 1))
    ypay_event = np.zeros((c_trans.shape[1], 1))
    value_noevent = np.zeros((c_trans.shape[1], 1))
    value_event = np.zeros((c_trans.shape[1], 1))
    ival_noevent = np.zeros((c_trans.shape[1], 1))
    ival_event = np.zeros((c_trans.shape[1], 1))
    dwght_noevent = np.zeros((c_trans.shape[1], 1))
    dwght_event = np.zeros((c_trans.shape[1], 1))
    salwght_noevent = np.zeros((c_trans.shape[1], 1))
    salwght_event = np.zeros((c_trans.shape[1], 1))
    valuex = np.zeros((c_trans.shape[1], 1))
    valuey = np.zeros((c_trans.shape[1], 1))
    iset = np.arange(0, c_trans.shape[1]).reshape(1, c_trans.shape[1])

    for i in np.arange(0, c_trans.shape[1]):
        pay_noevent[i, 0] = y_node[i, 0] * q_node[0, i] - c_trans[0, i] * q_node[0, i]  # payoff with no S&L event
        pay_event[i, 0] = y_node[i, 0] * q_node[0, i] - c_trans[0, i] * q_node[0, i] - y_node[i, 0] * \
                          q_node[0, i]  # payoff with S&L event
        xpay_noevent[i, 0] = pay_noevent[i, 0]  # payoff for route A with no S&L event
        xpay_event[i, 0] = pay_event[i, 0]  # payoff for route A with S&L event

    for i in np.arange(0, c_trans.shape[1]):
        inset = np.where(dtonei == dtonei[i, 0])[0]
        mask = np.not_equal(inset, i)
        ypay_noevent[i, 0] = np.mean([pay_noevent[j, 0] for j in range(len(mask)) if mask[j]])
        ypay_event[i, 0] = np.mean([pay_event[j, 0] for j in range(len(mask)) if mask[j]])
        value_noevent[i, 0] = np.abs(ypay_noevent[i, 0] - xpay_noevent[i, 0]) / (np.abs(ypay_noevent[i, 0]) +
                                                                                 np.abs(xpay_noevent[i, 0]) + 1)
        value_event[i, 0] = np.abs(ypay_event[i, 0] - xpay_event[i, 0]) / (np.abs(ypay_event[i, 0]) +
                                                                           np.abs(xpay_event[i, 0]) + 1)
        ipntlval = np.flip(np.argsort(np.array([value_noevent[i, 0], value_event[i, 0]])))
        ival_noevent[i, 0] = ipntlval[0]
        ival_event[i, 0] = ipntlval[1]
        dwght_noevent[i, 0] = (lccf ** ival_noevent[i, 0]) / ((lccf ** ival_noevent[i, 0]) * (1 - p_sl[0, i]) +
                                                              (lccf ** ival_event[i, 0]) * p_sl[0, i])
        dwght_event[i, 0] = (lccf ** ival_event[i, 0]) / ((lccf ** ival_noevent[i, 0]) * (1 - p_sl[0, i]) +
                                                          (lccf ** ival_event[i, 0]) * p_sl[0, i])
        salwght_noevent[i, 0] = (1 - p_sl[0, i]) * dwght_noevent[i, 0]
        salwght_event[i, 0] = p_sl[0, i] * dwght_event[i, 0]
        valuey[i, 0] = salwght_noevent[i, 0] * ypay_noevent[i, 0] + salwght_event[i, 0] * ypay_event[i, 0]
        valuex[i, 0] = salwght_noevent[i, 0] * xpay_noevent[i, 0] + salwght_event[i, 0] * xpay_event[i, 0]

    # Selection based on maximize profits while less than average S&L risk
    route = np.stack([np.multiply(np.transpose(rtpref), valuex)[:, 0].tolist(), np.transpose(p_sl)[:, 0].tolist(),
                      np.transpose(q_node)[:, 0].tolist(), np.transpose(iset)[:, 0].tolist(), dtonei[:, 0].tolist(),
                      np.transpose(totcpcty)[:, 0].tolist()], axis=1)
    rankroute = route[route[:, 0].argsort()[::-1]]

    # dtos = np.unique(dtonei[dtonei.any(axis=1)])
    dtos = np.unique(dtonei)
    icut = []

    if len(dtos) > 1:
        for j in np.arange(0, len(dtos)):
            idto = np.where(rankroute[:, 4] == dtos[j])[0]
            if np.where(valuex[np.where(dtonei == dtos[j])[0]] > 0)[0].size == 0:  # CHECK
                subicut = np.where(np.cumsum(rankroute[idto, 5]) >= totstock)[0][0:int(edgechange[j][0])][0]
            elif np.where(rankroute[idto, 1] > 0)[0].size == 0:
                subicut = np.where(rankroute[idto, 0] >= 0)[0][0:edgechange[j][0]]
            elif np.where(np.cumsum(rankroute[idto, 5]) >= totstock)[0].size == 0:
                subicut = np.where(rankroute[idto, 0] >= 0)[0][0:edgechange[j][0]]
            else:
                subicut = np.where(rankroute[idto, 0] >= 0)[0][0:edgechange[j][0]]
            if cutflag[int(dtos[j] - 1), 0] == 1:
                subicut = []
            # icut = np.concatenate((icut, idto[subicut]), axis=0)
            icut.append(idto[subicut])
    else:
        if len(np.where(valuex > 0)[0]) == 0:
            icut = np.transpose(np.arange(0, np.where(np.cumsum(rankroute[:, 5]) >= totstock)[0][0]))
        elif np.where(rankroute[:, 0] > 0)[0].size == 0:
            volcut = np.transpose(np.arange(0, np.where(np.cumsum(rankroute[:, 5]) >= totstock)[0][0]))
            valcut = np.where(rankroute[:, 0] >= 0)
            icut = np.isin(valcut, volcut)
        elif np.where(np.cumsum(rankroute[:, 5]) >= totstock)[0].size == 0:
            icut = np.where(rankroute[:, 0] >= 0)
        else:
            icut = np.where(rankroute[:, 0] >= 0)

    neipick = rankroute[icut, 3]
    neivalue = rankroute[icut, 0]

    return neipick, neivalue, valuex


def lldistkm(latlon1, latlon2):
    radius = 6371
    lat1 = latlon1[:, 0] * np.pi / 180
    lat2 = latlon2[:, 0] * np.pi / 180
    lon1 = latlon1[:, 1] * np.pi / 180
    lon2 = latlon2[:, 1] * np.pi / 180
    deltaLat = lat2 - lat1
    deltaLon = lon2 - lon1
    a = np.sin((deltaLat) / 2) ** 2 + np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.sin(deltaLon / 2) ** 2)
    c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d1km = np.multiply(radius, c)

    x = np.multiply(deltaLon, np.cos((lat1 + lat2) / 2))
    y = deltaLat
    d2km = np.multiply(radius, np.sqrt(np.multiply(x, x) + np.multiply(y, y)))

    return d1km, d2km


## Initilize.py

In [2]:
import numpy as np
import pandas as pd
import math


def load_expmntl_parms(ERUNS):
    empSLflag = np.zeros((1, ERUNS))  # determines is empirical (1) or artificial (0) S&L schedule used
    optSLflag = np.ones((1, ERUNS))
    suitflag = np.zeros((1, ERUNS))  # use RAT suitability (1) or build from covariates (0)

    sl_max = 125 * np.ones((1, ERUNS))  # baseline; maximum interdiction capacity

    sl_min = np.ceil(sl_max / 6)  # baseline; minimum interdiction capacity

    """
    CCDB year 2015 there were an average of 206 events (Carib and EP
    combined) per month, and a total volume of 1,592 MT. Assuming 25%
    underreporting in both events and volume, 258 events per month and a
    total volume of 1,990 MT or 166 MT per month. That would give an average
    capcaity of 644 kilos per movement per month.
    In 2015, 28%/72% of primary movements were in the Carib/East Pacific.
    Given the average monthly flow of 166 MT, 129,480 kg in EPac and 46,480 in
    Carib.
    With an average of 166MT per month and 156 over-land nodes, average
    capacity to handle off of that flow would be 1,071 kg
    """
    basecap = 32000 * np.ones((1, ERUNS))
    rtcap = np.array([[1, 1, 1, 1, 1, 1, 1.79, 2.55, 2.41, 2.01, 1.0, 1.03, 1.57, 2.06, 2.1, 3.07, 7.11, 6.44, 6.34],
                      [1, 1, 1, 1, 1, 1, 1.15, 1.4, 1.35, 1.2, 1.0, 1.31, 1.36, 1.46, 1.1, 1.13, 1.63, 1.49, 2.14]])
    low = np.linspace(0.1, 10, 6)
    high = np.linspace(10, 20, 6)
    p_sucintcpt = np.concatenate((low, high[1:6]))

    baserisk = 0.43 * np.ones((1, ERUNS))
    riskmltplr = 2 * np.ones((1, ERUNS))

    startstock = 16000 * np.ones((1, ERUNS))
    endstock = 292000 * np.ones((1, ERUNS))  # kg/month

    sl_learn = 0.6 * np.ones((1, ERUNS))  # baseline; rate of interdiction learning
    rt_learn = 0.3558 * np.ones((1, ERUNS))  # basline; rate of network agent learning

    losslim = 0.05 * np.ones((1, ERUNS))

    growthmdl = 2 * np.ones((1, ERUNS))
    prodgrow = 0.5 * np.ones((1, ERUNS))

    timewght = np.ones((1, ERUNS))  # time discounting for subjective risk perception (Gallagher, 2014), range[0,1.05]
    locthink = 0.35 * np.ones((1, ERUNS))  # 'Local thinker' coefficient for salience function (Bordalo et al., 2012) -
    # lower gives more weight to loss

    targetseize = 0.3417 * np.ones((1, ERUNS))  # baseline; target portion of total flow to seize

    intcpctymodel = np.ones((1, ERUNS))  # decreasing(1) or increasing(2) capacity response to missing target seizures

    profitmodel = np.ones((1, ERUNS))  # profit maximization model for node selection: 1 is standard, 2 is cumulative
    expandmax = 9 * np.ones((1, ERUNS))  # number of new nodes established per month

    return sl_max, sl_min, baserisk, riskmltplr, startstock, sl_learn, rt_learn, losslim, prodgrow, targetseize, \
        intcpctymodel, profitmodel, endstock, growthmdl, timewght, locthink, expandmax, empSLflag, optSLflag, \
        suitflag, rtcap, basecap, p_sucintcpt


# Interdiction Initialization #
def intrd_tables_batch(FLOW, slsuccess, SLPROB, NodeTable, EdgeTable, t, erun, mrun):
    Tflow = pd.DataFrame(columns=['End_Node', 'Start_Node', 'IntitFlow', 'DTO'], index=range(1, EdgeTable.shape[0]+1),
                         dtype=float)
    startFLOW = np.add(FLOW[:, :, t-1], slsuccess[:, :, t-1])
    Tintrd = pd.DataFrame(columns=['End_Node', 'Start_Node', 'IntitProb'], index=range(1, EdgeTable.shape[0]+1),
                          dtype=float)

    if t == 1:
        startSLPROB = SLPROB[:, :, 0]
    else:
        startSLPROB = SLPROB[:, :, t-2]

    sumprob = np.sum(startSLPROB)

    for i in range(EdgeTable.shape[0]):
        edge = EdgeTable.iloc[i]["EndNodes"]
        Tflow.iloc[i]["End_Node"] = edge[1]
        Tflow.iloc[i]["Start_Node"] = edge[0]
        Tflow.iloc[i]["IntitFlow"] = startFLOW[edge[0]][edge[1]]
        Tflow.iloc[i]["DTO"] = NodeTable.iloc[edge[1]]["DTO"]

        Tintrd.iloc[i]["End_Node"] = edge[1]
        Tintrd.iloc[i]["Start_Node"] = edge[0]
        Tintrd.iloc[i]["IntitProb"] = startSLPROB[edge[0]][edge[1]] / sumprob

    t1 = int(t >= 100)

    if t >= 100:
        t2 = math.floor((t - 100) / 10)
    else:
        t2 = math.floor(t / 10)

    mrun_t1 = math.floor(mrun / 10)
    mrun_t2 = mrun % 10
    erun_t1 = math.floor(erun/100)
    erun_t2 = math.floor(erun/10)
    erun_t3 = erun % 10

    # Saving to excel required?
    # Tflow.to_excel('../FunctionTesting/Tflow_python.xlsx')
    # Tintrd.to_excel('../FunctionTesting/Tintrd_python.xlsx')

    return Tflow, Tintrd


## Optimize.py

In [3]:
"""   Interdiction events from optimization model   """

import numpy as np
import math


def optimize_interdiction_batch(ADJ):
    trgtfile = r'C:\Users\htanveer\Narc\NarcoABM_python\data\MTMCI_IntNodes.txt'

    Tintevent = np.loadtxt(trgtfile, dtype=int)
    intrdct_events = np.zeros(ADJ.shape)
    intrdct_nodes = Tintevent
    for j in np.arange(0, len(Tintevent)):
        iupstream = (ADJ[:, Tintevent[j] - 1] == 1)
        intrdct_events[iupstream, Tintevent[j] - 1] = 1

    return intrdct_events, intrdct_nodes


# Top-down supply chain optimization ########
def optimizeroute_multidto(dtorefvec, subflow, supplyfit, expmax, subroutepref, dtoEdgeTable, dtoSLRISK, dtoADDVAL,
                           dtoCTRANS, losstolval, dtoslsuc):
    iactiveedges = np.concatenate((np.where(subflow > 0), np.where(dtoslsuc > 0)), axis=1)
    edgeparms = []
    for edge in range(len(iactiveedges[0])):
        edgeparms.append(np.array([subflow[iactiveedges[0][edge], iactiveedges[1][edge]],
                                   dtoSLRISK[iactiveedges[0][edge], iactiveedges[1][edge]], iactiveedges[0][edge],
                                   iactiveedges[1][edge]]))
    edgeparms = np.array(edgeparms)

    if supplyfit < losstolval:  # need to consolidate supply chain
        edgesort = edgeparms[edgeparms[:, 1].argsort()[::-1]]
        # primary movement
        iprimary = list(np.intersect1d(np.where(edgesort[:, 2] == 0)[0],
                                       np.where(edgesort[:, 3] != len(dtorefvec) - 1)[0]))
        upper_lim = min(round(len(iactiveedges[0]) * (supplyfit / (supplyfit + losstolval))), len(iactiveedges[0]) - 1)
        if upper_lim > 0:
            edgecut = np.arange(0, upper_lim)
        else:
            edgecut = []

        # Preserve at least one primary movement
        minrisk_primary = np.amin(edgesort[iprimary, 1])
        ikeep_primary = np.where(edgesort[iprimary, 1] == minrisk_primary)[0]
        if len(ikeep_primary) != 1:
            maxprofit_primary = max(edgesort[iprimary[ikeep_primary], 0])
            ikeep_primary = ikeep_primary[edgesort[iprimary[ikeep_primary], 0] == maxprofit_primary]
            if len(ikeep_primary) != 1:
                ikeep_primary = ikeep_primary[0]

        if len(edgecut) > 0:
            edgecut = np.delete(edgecut, np.intersect1d(edgecut, [iprimary[ikeep_primary[0]]] +
                                                        list(np.where(edgesort[edgecut, 2] ==
                                                                      edgesort[iprimary[ikeep_primary[0]], 3])[0])))

        # remove highest risk edges
        for j in range(0, len(edgecut)):
            icheckroute = np.where(subflow(edgesort[edgecut[j], 3],
                                           np.intersect1d(dtorefvec, dtoEdgeTable['EndNodes'].str[1]
                                           [np.where(dtoEdgeTable['EndNodes'].str[0] ==
                                                     dtorefvec[edgesort[edgecut[j], 3]])[0]]))
                                   > 0)
            actroutes = dtoEdgeTable['EndNodes'].str[1][np.where(dtoEdgeTable['EndNodes'].str[0] ==
                                                                 dtorefvec[edgesort[edgecut[j], 3]])[0]]
            checknoderoutes = (
                    len(actroutes[icheckroute]) == len(np.where(edgesort[edgecut, 3] == edgesort[edgecut[j], 3])))
            if checknoderoutes:
                cutsenders = np.where(dtorefvec[np.in1d(dtorefvec,
                                                        dtoEdgeTable['EndNodes'].str[0]
                                                        [np.where(dtoEdgeTable['EndNodes'].str[1] ==
                                                                  dtorefvec[edgesort[edgecut[j], 3]])[0]])]
                                      == 1)
                for i in range(len(cutsenders)):
                    subroutepref[cutsenders[i], edgesort[edgecut[j], 3]] = 0

            if len(icheckroute) == 1:
                subroutepref[edgesort[edgecut[j], 2]] = 0
                irmvsender = (edgesort[:, 4] == edgesort[edgecut[j], 3])
                subroutepref[edgesort[irmvsender, 2]] = 0
            else:
                subroutepref[edgesort[edgecut[j], 2]] = 0

    elif supplyfit >= losstolval:  # need to expand supply chain
        potnodes = np.delete(dtorefvec, np.in1d(dtorefvec, np.concatenate(([1], dtorefvec[np.int_(
            np.unique(edgeparms[:, 2:4]))].flatten())))).reshape(-1, 1)
        edgeadd = np.arange(0, min(max(math.ceil(supplyfit / losstolval), 1), min(expmax, len(potnodes))))

        if len(np.where(potnodes)[0]) == 0:
            pass
        else:
            newedgeparms = []
            potsenders = np.int_(np.unique(edgeparms[:, 2:4]))  # dto node index
            potsenders = potsenders[potsenders != len(dtorefvec)]
            for k in range(0, len(potsenders)):
                ipotreceive = np.where(np.in1d(potnodes,
                                               dtoEdgeTable['EndNodes'].str[1][np.where(dtoEdgeTable['EndNodes'].str[0]
                                                                                        == dtorefvec[potsenders[k], 0])
                                               [0]]) == 1)[0]

                if len(np.where(ipotreceive)[0]) == 0:
                    continue
                ipotedge_col = np.where(np.in1d(dtorefvec, potnodes[ipotreceive]) == 1)[0]
                for i in range(len(ipotedge_col)):
                    newedgeparms.append([(dtoADDVAL[potsenders[k], ipotedge_col[i]] -
                                          dtoCTRANS[potsenders[k], ipotedge_col[i]]),
                                         dtoSLRISK[potsenders[k], ipotedge_col[i]], potsenders[k], ipotedge_col[i],
                                         ipotreceive[i]])
            newedgeparms = np.array(newedgeparms)
            edgesort = newedgeparms[newedgeparms[:, 0].argsort()[::-1]]
            subroutepref[np.int_(edgesort[edgeadd, 2]), np.int_(edgesort[edgeadd, 3])] = 1
            rows = np.in1d(dtoEdgeTable['EndNodes'].str[0], dtorefvec[np.int_(edgesort[edgeadd, 3])])
            ireceivers = [dtoEdgeTable.loc[row, 'EndNodes'] for row in range(len(rows)) if rows[row]]
            ireceivers = np.array(ireceivers)
            send_row = []
            rec_col = []
            for jj in range(0, len(ireceivers[:, 0])):
                send_row = np.append(send_row, np.where(np.in1d(dtorefvec, ireceivers[jj, 0]) == 1)[0])
                rec_col = np.append(rec_col, np.where(np.in1d(dtorefvec, ireceivers[jj, 1]) == 1)[0])

            subroutepref[np.int_(send_row), np.int_(rec_col)] = 1
            subroutepref[np.int_(rec_col), len(dtorefvec)] = 1

    return subroutepref


## data.py

In [ ]:
nodes_testing = data_sourcing()

In [ ]:
nodes_testing.to_csv("NodeInfo_gdb.csv")

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import gurobipy as gp


def data_sourcing():
    nodes_fp = r'C:\Users\htanveer\Narc\Code\NarcoLogic\TrialResults\Nodes_2.csv'
    if os.path.exists(nodes_fp):
        nodes = pd.read_csv(nodes_fp)
        nodes.drop(nodes.columns[[0]], axis=1, inplace=True)
        print('Nodes_2 found!')
    else:
        nodes = pd.DataFrame.spatial.from_featureclass(
            r'C:\Users\htanveer\Narc\DISN.gdb\IllicitNetworks.gdb\NodeInfo_cln')
        nodes.drop(['OBJECTID'], axis=1, inplace=True)
        nodes['Timestep'] = 1
    return nodes


def data_processing(Tflow, i, mr):
    Nodes = data_sourcing()
    strt = time.time()
    # cast the returned TFlow table and timestep to py variables
    # input will be tflow for t = n x mr 1-n; need to group t by model run; doublecheck the tflow output
    tf2 = Tflow
    NLtimestep = i

    # convert object type column values in TFlow to float by first converting to string and omitting special characters
    tf2['End_Node'] = tf2['End_Node'].astype(str)
    tf2['End_Node'] = tf2['End_Node'].str.replace(r'[(),]', "")
    tf2['End_Node'] = tf2['End_Node'].astype(float)
    tf2['Start_Node'] = tf2['Start_Node'].astype(str)
    tf2['Start_Node'] = tf2['Start_Node'].str.replace(r'[(),]', "")
    tf2['Start_Node'] = tf2['Start_Node'].astype(float)
    tf2['IntitFlow'] = tf2['IntitFlow'].astype(str)
    tf2['IntitFlow'] = tf2['IntitFlow'].str.replace(r'[(),]', "")
    tf2['IntitFlow'] = tf2['IntitFlow'].astype(float).round(2)
    tf2['DTO'] = tf2['DTO'].astype(str)
    tf2['DTO'] = tf2['DTO'].str.replace(r'[(),]', "")
    tf2['DTO'] = tf2['DTO'].astype(float)  # can't be int bc there are 0 values
    tf2['MR'] = mr

    # combine Nodes with tf2 (tflow table) and write to new df
    Nodes_2 = Nodes.merge(tf2, on=['End_Node'], how='left')
    ActiveLinks = Nodes.merge(tf2, on=['End_Node'], how='left')

    # Get the Type value by node stored as a dict by ID e.g. {1: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    # 2: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]}
    ID = Nodes_2['ID']
    Types = Nodes_2.filter(regex='^Type', axis=1)
    ID_Type = Types.join(ID, how='right')
    ID_TypeDict = ID_Type.set_index('ID').T.to_dict('list')

    # Sum the positive type values by ID e.g. {2: 1.0, 3: 1.0, 4: 2.0,...} to divide the updated flow by per node.
    SumType = {k: sum(v) for k, v in ID_TypeDict.items()}

    # convert TypeSum dict to column
    Nodes_2['SumType'] = Nodes_2['ID'].map(SumType)

    # Read Interdicted Nodes text file to list; write to comp IntNodes
    with open(r'C:\Users\htanveer\Narc\Code\NarcoLogic\MTMCI_IntNodes\MTMCI_IntNodes.txt', "r") as f:
        IntList = [int(x) for x in f.read().split()]

    # Aggregate flows from NarcoLogic by End Node, count active number of trafficking nodes (node degree)
    # drop start nodes and individual link flow values--everything based on end node now, summed into tFlow
    nd = Nodes_2.groupby("ID")["IntitFlow"].agg(NodeDegree=lambda x: x[x != 0].count(), NLFlow="sum")
    Nodes_2 = pd.merge(Nodes_2, nd, how='inner', on='ID')
    Nodes_2.drop(['Start_Node'], axis=1, inplace=True)
    Nodes_2.drop_duplicates(subset=['ID'], keep='last', inplace=True)

    # Updated flow values need to be updated on Nodes_2 tble, but XNodes need to be on previous Nodes table.
    # Perform the 25% rule (.75 * expected flow value (PMNode) + .25 * actual flow (NLFlow) =  PMNode2
    # If the node is an interdicted node do: (.75 * expected flow value (PMNode) + .25 * actual flow (NLFlow) =  PMNode2
    # If not interdicted, reduce expected flow (25% rule);
    Nodes_2['PMNode2'] = np.where(Nodes_2['ID'].isin(IntList), ((Nodes_2['PMNode'] * .75) + (Nodes_2['NLFlow'] * .25)),
                                  (Nodes_2['PMNode'] * .75))

    # Update Flow column--calculate the updated flow value from PMNode2 to distribute to the positive ait columns
    # divide the updated flow value 'PMNode_2' by number of nodes w/ Type = 1 ('SumType')
    Nodes_2.loc[Nodes_2.SumType != 0, 'UpdateFlow'] = (Nodes_2['PMNode2'] / Nodes_2['SumType'])
    Nodes_2.loc[Nodes_2.SumType == 0, 'UpdateFlow'] = (Nodes_2['PMNode2'])

    # allocate the Update Flow to positive ait/type values
    TypeNums = [1, 2, 3, 4, 5, 6, 7, 8]
    for num in TypeNums:
        Nodes_2['ait{}'.format(num)] = Nodes_2['Type{}'.format(num)] * Nodes_2['UpdateFlow']

    # Update Timestep
    Nodes_2['Timestep'] = NLtimestep
    ActiveLinks['Timestep'] = NLtimestep

    # Overwrite PMNode with updated perceived interdiction flow vales (PMNode2),
    # Remove Unnecessary Columns, fill NaNs w 0
    Nodes_2['PMNode'] = Nodes_2['PMNode2']
    Nodes_2.drop(['IntitFlow', 'SumType', 'PMNode2', 'UpdateFlow', 'SHAPE'], axis=1, inplace=True)
    Nodes_2.fillna(0, inplace=True)

    # Drop unnecessary columns in Active Links, omit 0 Flow rows, reset index
    ActiveLinks.drop(ActiveLinks.columns[ActiveLinks.columns.str.contains('ait|Type|Country|PMNode|SHAPE')], axis=1,
                     inplace=True)
    ActiveLinks.drop(ActiveLinks.loc[ActiveLinks['IntitFlow'] == 0].index, inplace=True)
    ActiveLinks.dropna(inplace=True)
    ActiveLinks.reset_index(drop=True, inplace=True)

    # write active links to table; concat w/ each timestep
    CompActLinksFP = r'C:\Users\htanveer\Narc\Code\NarcoLogic\TrialResults\CompActLinks.csv'  # this filename will
    # need a MR suffix
    if os.path.exists(CompActLinksFP):
        CompActLinks = pd.read_csv(CompActLinksFP)  # drop index
        CompActLinks.drop(CompActLinks.columns[0], axis=1, inplace=True)  # omit this line
        # write XNodes to CompActLinks
        ComprehensiveActLinks = pd.concat([CompActLinks, ActiveLinks])
        ComprehensiveActLinks.to_csv(CompActLinksFP)

    else:
        ActiveLinks.to_csv(CompActLinksFP)

    # Concatinate Nodes_2 table with initial Node table if TS 1->2, otherwise Concat CompDF with Nodes_2 and overwrite
    # CompDF csv

    CompFP = r'C:\Users\htanveer\Narc\Code\NarcoLogic\TrialResults\ComprehensiveNodesDF.csv'
    # this is t2 and up. This finds the existing CompDF table and concats the latest results (nodes2) to it.
    if os.path.exists(CompFP):
        Comp = pd.read_csv(CompFP)  # read this in w/o index
        Comp.drop(Comp.columns[0], axis=1, inplace=True)

        # IMPT! Add interdicted Nodes from txt file to PREVIOUS timestep node info df (*Comp df*)
        XNodes = pd.read_csv(r'C:\Users\htanveer\Narc\Code\NarcoLogic\MTMCI_IntNodes\MTMCI_IntNodes.txt',
                             header=None)
        XNodes.rename(columns={0: 'ID'}, inplace=True)
        XNodes['Values'] = '1'
        IntDict = dict(zip(XNodes['ID'], XNodes['Values']))
        # add filter for MR as well
        filt = Comp['Timestep'] == i
        Comp.loc[filt, ['XNodes']] = Comp['ID'].map(IntDict)
        Comp.fillna({'XNodes': 0}, inplace=True)

        # Then add Nodes2 (latest data) on to CompDf using concat
        ComprehensiveNodesDF = pd.concat([Comp, Nodes_2])
        ComprehensiveNodesDF.to_csv(CompFP)
    else:
        # this is t 1 to t 2 transition which writes the first CompDF table
        # Add interdicted Nodes from txt file to PREVIOUS timestep Nodes table (*Nodes df*)
        XNodes = pd.read_csv(r'C:\Users\htanveer\Narc\Code\NarcoLogic\MTMCI_IntNodes\MTMCI_IntNodes.txt',
                             header=None)
        XNodes.rename(columns={0: 'ID'}, inplace=True)
        XNodes['Values'] = '1'
        IntDict = dict(zip(XNodes['ID'], XNodes['Values']))
        filt = Nodes['Timestep'] == i
        Nodes.loc[filt, ['XNodes']] = Nodes['ID'].map(IntDict)
        Nodes.fillna({'XNodes': 0}, inplace=True)

        # Concat Nodes with Nodes2
        ComprehensiveNodesDF1 = pd.concat([Nodes, Nodes_2])
        ComprehensiveNodesDF1.to_csv(CompFP)

    # Make Nodes_2 like Nodes 1; Nodes_2 drop DTO NodeDegree Tflow Update Flow after its written to CompDF
    Nodes_2.drop(['DTO', 'NodeDegree', 'NLFlow', 'MR'], axis=1, inplace=True)

    # Write Nodes_2 to csv if NLtimestep 1 to 179; remove if t 180 so t 1, m:n can resume w Nodes feature
    Nodes_FP = r'C:\Users\htanveer\Narc\Code\NarcoLogic\TrialResults\Nodes_2.csv'
    # change this value to whatever the end of the NL range is e.g. 180
    if NLtimestep < 180:
        Nodes_2.to_csv(Nodes_FP)
    else:
        os.remove(Nodes_FP)

    #     NodesFP_i= r'C:\Users\pcbmi\Box\NSF_D-ISN\Code\NarcoLogic\TrialResults\Nodes_2Final_t{}.csv'.format
    #     (NLtimestep)
    #     Nodes_2.to_csv(NodesFP_i)

    print('Data Processing for timestep {} complete!'.format(NLtimestep))
    elapsed_time = time.time() - strt
    print('DP Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))


def MTMCI_func(NodesDF, timestep, mr):
    """Writes Interdicted Nodes as text file."""
    MTMCLP = gp.Model()
    MTMCLP.setParam('OutputFlag', 0)
    x = {}
    y = {}

    I = NodesDF['ID'].tolist()
    J = NodesDF['ID'].tolist()
    T = range(1, 9)

    for j in J:
        for t in T:
            x[j, t] = MTMCLP.addVar(vtype=gp.GRB.BINARY, name=f'Node{j, t}')
    for i in I:
        for t in T:
            y[i, t] = MTMCLP.addVar(vtype=gp.GRB.BINARY, name=f'Demand{i, t}')

    # Create a[i,t] dictionary
    # make a dataframe with just the ait# and ID field
    Ait = NodesDF.filter(regex='^ait', axis=1)
    ID = NodesDF['ID']
    ID_Ait = Ait.join(ID, how='right')

    # Dataframe to MultiIndex TupleDict e.g. {(i,t):a, (i,t):a...}
    # stripping 'ait' string from list, converting to int
    IDs = ID_Ait.ID.tolist()
    others = list(ID_Ait.columns)
    others.remove("ID")
    others_t = list(map(lambda st: str.replace(st, 'ait', ''), others))
    t_int = list(map(lambda ele: int(ele), others_t))
    index_tuples = [(ID, other) for ID in IDs for other in t_int]
    multi_ix = pd.MultiIndex.from_tuples(index_tuples)
    df1 = pd.DataFrame(ID_Ait[others].values.ravel(), index=multi_ix, columns=["data"])
    a = df1.to_dict()["data"]

    # Create U[j] lists for each node, list the Type#s that have 0 as a value for that ID#
    # e.g. {5:[1,2,4,5,6,7,8]}
    Uj = NodesDF.filter(regex='^Type', axis=1)
    Uj_ID = Uj.join(ID, how='right')
    U = {k: [int(i[-1]) for i, v in d.items() if v == 0]
         for k, d in Uj_ID.set_index('ID').to_dict('index').items()}

    # KMC/formulation version
    for j in J:
        MTMCLP.addConstr(gp.quicksum(x[j, t] for t in U[j]), gp.GRB.EQUAL, 0)

    # #Cov Constraint w/o quicksum for j in J (3) #loc i can be covered >1 if by diff FP type (Const 3c)
    for i in I:
        for t in T:
            MTMCLP.addConstr(x[i, t], gp.GRB.GREATER_EQUAL, y[i, t], f'Covering Constraint{i, t}')

    # Locate Number of Force Packages (4)
    P = {1: 3, 2: 2, 3: 4, 4: 2, 5: 3, 6: 4, 7: 1, 8: 2}

    for t in T:
        MTMCLP.addConstr(gp.quicksum(x[j, t] for j in range(1, 164)), gp.GRB.EQUAL, P[t], f'Type Force Packages{j, t}')

    ##Set the Objective Function (1)
    MTMCLP.setObjective(gp.quicksum(a[i, t] * y[i, t] for i in I for t in T), gp.GRB.MAXIMIZE)

    ##Update
    MTMCLP.update()
    MTMCLP.optimize()

    intSites = []
    for j, t in x:
        if x[j, t].X == 1:
            intSites.append(str(j))
            # print(f'Node {j} | Type {t}')

    obj = MTMCLP.getObjective()
    print("Objective Value: " + str(obj.getValue()))

    # write interdicted nodes to text file
    SolutionFP = r'C:\Users\htanveer\Narc\Code\NarcoLogic\MTMCI_IntNodes\MTMCI_IntNodes.txt'

    with open(SolutionFP, 'w') as q:
        for site in intSites:
            q.write('%s\n' % site)

    print("MTMCLP for timestep {}, model run {} is complete.".format(timestep, mr))

## Narcologic.py

In [ ]:
from scipy import io
# from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [ ]:
 # Ignore divide by zero runtime warning
np.seterr(divide='ignore')

ERUNS = 11
TSTART = 0
TMAX = 181  # 15 years at monthly time steps

# thistate = scipy.io.loadmat('data/savedrngstate.mat')['thistate']

erun = 3
mrun = 2

# Start initialization, set random number generator state for repeatability

np.random.seed(mrun)

# load experimental parameters file
sl_max, sl_min, baserisk, riskmltplr, startstock, sl_learn, rt_learn, losslim, prodgrow, targetseize, \
    intcpctymodel, profitmodel, endstock, growthmdl, timewght, locthink, expandmax, empSLflag, optSLflag, suitflag, \
    rtcap, basecap, p_sucintcpt = load_expmntl_parms(ERUNS)

# Load landscape files
dcoast = scipy.io.loadmat(r'C:\Users\htanveer\Narc\NarcoABM_python\data\coast_dist')['dcoast']
LANDSUIT = scipy.io.loadmat(r'C:\Users\htanveer\Narc\NarcoABM_python\data\landsuit_file_default')['LANDSUIT']

# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# @@@@@@@@@@ Agent Attributes @@@@@@@@@@@@
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

# Network Agent #
ndto = 2  # initial number of DTOs
dtocutflag = np.zeros((ndto, 1))
DTOBDGT = np.zeros((ndto, TMAX))
losstol = losslim[0, erun]  # tolerance threshold for loss due to S&L, triggers route fragmentation
stock_0 = startstock[0, erun]  # initial cocaine stock at producer node
stock_max = endstock[0, erun]
startvalue = 4500  # producer price, $385/kg: Zoe's numbers 4,500 in Panama
deltavalue = 4.46  # added value for distance traveled $8/kilo/km: Zoe's numbers $4.46
nodeloss = 0  # amount of cocaine that is normally lost (i.e., non-interdiction) at each node
ctrans_inland = 371  # transportation costs (kg/km) over-ground (3.5), includes
ctrans_coast = 160  # transportation costs (kg/km) via plane or boat (1.5)
ctrans_air = 3486
delta_rt = rt_learn[0, erun]  # reinforcement learning rate for network agent

# (i.e., weight on new information for successful routes)
# perceived risk model
alpharisk = 2
betarisk = 0.5
timewght_0 = timewght[0, erun]

slprob_0 = 1 / (np.sum(np.power(timewght_0, np.array((np.arange(0, 13))))) + betarisk)
bribepct = 0.3
rentcap = 1 - bribepct
edgechange = expandmax[0, erun] * np.ones((ndto, 1))

savedState = np.random.get_state()  # CHECK
# np.random.seed(thistate)

###################################################################
#   Build trafficking network - NodeTable and EdgeTable   #
###################################################################

EdgeTable = pd.read_csv(r'C:\Users\htanveer\Narc\NarcoABM_python\data\EdgeTable.csv')
EdgeTable['EndNodes'] = EdgeTable[['EndNodes_1', 'EndNodes_2']].values.tolist()
EdgeTable = EdgeTable.drop(columns=['EndNodes_1', 'EndNodes_2'])
NodeTable = pd.read_csv(r'C:\Users\htanveer\Narc\NarcoABM_python\data\NodeTable.csv')
NodeTable['Row'] = NodeTable['Row'] - 1
NodeTable['Col'] = NodeTable['Col'] - 1
EdgeTable['Capacity'] = rtcap[0, erun] * np.ones(EdgeTable.shape[0])
nnodes = NodeTable.shape[0]
mexnode = nnodes - 1
endnodeset = mexnode
coastdist = [dcoast[row][col] for row, col in zip(NodeTable['Row'], NodeTable['Col'])]
NodeTable['CoastDist'] = coastdist
NodeTable['CoastDist'][0] = 0
NodeTable['CoastDist'][nnodes - 1] = 0

# Assign nodes to initials DTOs
for nn in range(1, nnodes - 2):
    try:
        westdir = NodeTable['Col'][nn] - np.where(np.isnan(dcoast[NodeTable['Row'][nn],
        np.arange(0, NodeTable['Col'][nn] - 1)]) == 1)[0][-1]
    except IndexError:
        westdir = 0
    try:
        eastdir = np.where(np.isnan(dcoast[NodeTable['Row'][nn], np.arange(NodeTable['Col'][nn],
                                                                           LANDSUIT.shape[1] - 1)]) == 1)[0][0]
    except IndexError:
        eastdir = 0

    if westdir < 2.5 * eastdir:
        NodeTable['DTO'][nn] = 1
    else:
        NodeTable['DTO'][nn] = 2

ADJ = np.zeros((nnodes, nnodes))  # adjacency matrix for trafficking network
DIST = np.zeros((nnodes, nnodes))  # geographic distance associated with edges
ADDVAL = np.zeros((nnodes, nnodes))  # added value per edge in trafficking network
WGHT = np.ones((nnodes, nnodes))  # dynamic weighting of edges
SLRISK = slprob_0 * np.ones((nnodes, nnodes))  # dynamic perceived risk of seisure and loss per edge by node agent
CPCTY = np.zeros((nnodes, nnodes))  # maximum flow possible between nodes
CTRANS = np.zeros((nnodes, nnodes, TMAX))  # ransportation costs between nodes
RMTFAC = np.zeros((nnodes, nnodes))  # landscape factor (remoteness) influencing S&L risk
COASTFAC = np.zeros((nnodes, nnodes))  # landscape factor (distance to coast) influencing S&L risk
LATFAC = np.zeros((nnodes, nnodes))  # decreased likelihood of S&L moving north to reflect greater DTO investment
BRDRFAC = np.zeros((nnodes, nnodes))  # increased probability of S&L in department bordering an
# international border

SUITFAC = np.zeros((nnodes, nnodes))
STOCK = np.zeros((nnodes, TMAX))  # dynamic cocaine stock at each node
PRICE = np.zeros((nnodes, TMAX))  # $/kilo at each node
RISKPREM = np.ones((nnodes, nnodes, TMAX))
OUTFLOW = np.zeros((nnodes, TMAX))  # dynamic stock of cocaine leaving from each node
TOTCPTL = np.zeros((nnodes, TMAX))  # total value of cocaine at each node
ICPTL = np.zeros((nnodes, TMAX))  # dynamic illicit capital accumulated at each node
BRIBE = np.zeros((nnodes, TMAX))  # annual bribe payments made at each node to maintain control
MARGIN = np.zeros((nnodes, TMAX))  # gross profit per node after purchasing, trafficking, and selling
RENTCAP = np.zeros((nnodes, TMAX))  # portion of MARGIN retained at node as profit
LEAK = np.zeros((nnodes, TMAX))  # dynamic amount of cocaine leaked at each node
activeroute = np.empty((nnodes, TMAX), dtype=object)  # track active routes
avgslrisk = np.empty((nnodes, TMAX), dtype=object)  # average S&L risk at each node given active routes
totslrisk = np.zeros((1, TMAX))  # network-wide average S&L risk
slcpcty = np.zeros((1, TMAX))

np.random.set_state(savedState)

for k in range(0, nnodes):
    # Create adjacency matrix
    ADJ[k, EdgeTable['EndNodes'].str[1][np.where(EdgeTable['EndNodes'].str[0] == k)[0]]] = 1

# Node Attributes
remotefac = (1 - NodeTable['PopSuit'].to_numpy()).reshape(-1, 1)
remotefac[0, 0] = 0
brdrfac = NodeTable['DistBorderSuit'].to_numpy().reshape(-1, 1)
brdrfac[0, 0] = 0
suitfac = NodeTable['LandSuit'].to_numpy().reshape(-1, 1)
suitfac[0, 0] = 0
coastfac = (NodeTable['CoastDist'].to_numpy() / np.amax(NodeTable['CoastDist'])).reshape(-1, 1)
coastfac[0, 0] = 0
nwvec = (np.sqrt(np.multiply(0.9, NodeTable['Lat'].to_numpy() ** 2) +
                 np.multiply(0.1, NodeTable['Lon'].to_numpy() ** 2))).reshape(-1, 1)
latfac = 1 - nwvec / np.amax(nwvec)
latfac[0, 0] = 0

# Create adjacency matrix
iendnode = NodeTable.loc[NodeTable['DeptCode'] == 2, 'ID'].iloc[0]
ADJ[EdgeTable['EndNodes'].str[0][np.where(EdgeTable['EndNodes'].str[1] == iendnode)[0]], iendnode] = 1

for j in range(0, nnodes):
    # Create weight and capacity matrices
    WGHT[0, np.where(ADJ[j, :] == 1)[0]] = EdgeTable['Weight'][np.where(ADJ[j, :] == 1)[0]]
    CPCTY[j, np.where(ADJ[j, :] == 1)[0]] = EdgeTable['Capacity'][np.where(ADJ[j, :] == 1)[0]]
    # Create distance (in km) matrix
    rows = [int(x) for x in range(len(ADJ[j, :])) if ADJ[j, x] == 1]
    latlon2 = NodeTable.loc[rows, ['Lat', 'Lon']].to_numpy()

    latlon1 = np.tile(NodeTable.loc[j, ['Lat', 'Lon']].to_numpy(), (len(latlon2[:, 1]), 1))
    d1km, d2km = lldistkm(latlon1, latlon2)
    DIST[j, np.where(ADJ[j, :] == 1)[0]] = d1km

    # Create added value matrix (USD) and price per node
    ADDVAL[j, np.where(ADJ[j, :] == 1)[0]] = np.multiply(deltavalue, DIST[j, np.where(ADJ[j, :] == 1)[0]])
    if j == 0:
        PRICE[j, TSTART] = startvalue
    elif j == endnodeset:
        continue
    elif 156 <= j <= 159:
        isender = EdgeTable['EndNodes'].str[0][np.where(EdgeTable['EndNodes'].str[1] == j)[0]]
        inextleg = EdgeTable['EndNodes'].str[1][np.where(EdgeTable['EndNodes'].str[0] == j)[0]]
        PRICE[j, TSTART] = PRICE[isender, TSTART] + ADDVAL[isender, j] + PRICE[isender, TSTART] + np.mean(
            ADDVAL[j, inextleg])
        # even prices for long haul routes
        if j == 159:
            PRICE[np.array([156, 159]), TSTART] = np.amin(PRICE[np.array([156, 159]), TSTART])
            PRICE[np.array([157, 158]), TSTART] = np.amin(PRICE[np.array([157, 158]), TSTART])
    else:
        isender = EdgeTable['EndNodes'].str[0][EdgeTable['EndNodes'].str[1] == j]
        PRICE[j, TSTART] = np.mean(PRICE[isender, TSTART] + ADDVAL[isender, j])

    """
    loop not needed as endnodeset is an integer
    for en in range(0, len(endnodeset)+1):
        PRICE[endnodeset[en], TSTART] = np.amax(PRICE[np.where(ADJ[:, endnodeset[en]] == 1)[0], TSTART])
    """
    PRICE[endnodeset, TSTART] = np.amax(PRICE[np.where(ADJ[:, endnodeset] == 1)[0], TSTART])
    RMTFAC[j, np.where(ADJ[j, :] == 1)[0]] = remotefac[np.where(ADJ[j, :] == 1)[0]].flatten()
    COASTFAC[j, np.where(ADJ[j, :] == 1)[0]] = coastfac[np.where(ADJ[j, :] == 1)[0]].flatten()
    LATFAC[j, np.where(ADJ[j, :] == 1)[0]] = latfac[np.where(ADJ[j, :] == 1)[0]].flatten()
    BRDRFAC[j, np.where(ADJ[j, :] == 1)[0]] = brdrfac[np.where(ADJ[j, :] == 1)[0]].flatten()
    SUITFAC[j, np.where(ADJ[j, :] == 1)[0]] = suitfac[np.where(ADJ[j, :] == 1)[0]].flatten()

    # Transportation costs
    ireceiver = (EdgeTable['EndNodes'].str[1][np.where(EdgeTable['EndNodes'].str[0] == j)[0]]).to_numpy()
    ireceiver = ireceiver.reshape(len(ireceiver), 1)
    idist_ground = np.logical_and(DIST[j, ireceiver] > 0, DIST[j, ireceiver] <= 500).reshape(-1, 1)
    idist_air = (DIST[j, ireceiver] > 500).reshape(-1, 1)
    """ CHECK FOR DIVIDE BY ZERO for CTRANS calculations """
    CTRANS[j, ireceiver[idist_ground], TSTART] = np.multiply(ctrans_inland,
                                                             DIST[j, ireceiver[idist_ground]]) / DIST[0, mexnode]
    CTRANS[j, ireceiver[idist_air], TSTART] = np.multiply(ctrans_air,
                                                          DIST[j, ireceiver[idist_air]]) / DIST[0, mexnode]

    if NodeTable.loc[j, 'CoastDist'] < 20 or 156 <= j <= 158:
        ireceiver = (EdgeTable['EndNodes'].str[1][EdgeTable['EndNodes'].str[0] == j]).to_numpy()
        ireceiver = ireceiver.reshape(len(ireceiver), 1)
        idist_coast = (NodeTable.loc[ireceiver[:, 0], 'CoastDist'] < 20).to_numpy().reshape(-1, 1)
        CTRANS[j, ireceiver[idist_coast], TSTART] = np.multiply(ctrans_coast,
                                                                DIST[j, ireceiver[idist_coast]]) / DIST[0, mexnode]
        if 156 <= j <= 158:
            CTRANS[j, ireceiver[idist_coast], TSTART] = 0
            CTRANS[0, j, TSTART] = CTRANS[0, j, TSTART] + np.mean(
                np.multiply(ctrans_coast, DIST[j, ireceiver[idist_coast]]) / DIST[1, mexnode])

# Initialize Interdiction agent
# Create S&L probability layer
routepref = np.zeros((nnodes, nnodes, TMAX))  # weighting by network agent of successful routes
slevent = np.zeros((nnodes, nnodes, TMAX))  # occurrence of S&L event
intrdctobs = np.zeros((nnodes, nnodes, TMAX))
slnodes = [[] for _ in range(TMAX)]
slsuccess = np.zeros((nnodes, nnodes, TMAX))  # volume of cocaine seized in S&L events
slvalue = np.zeros((nnodes, nnodes, TMAX))  # value of cocaine seized in S&L events
slcount_edges = np.zeros((1, TMAX))
slcount_vol = np.zeros((1, TMAX))
INTRDPROB = np.zeros((nnodes, TMAX))
SLPROB = np.zeros((nnodes, nnodes, TMAX))  # dynamic probability of S&L event per edge

facmat_list = [LATFAC, COASTFAC, RMTFAC, DIST / np.amax(np.amax(DIST)), BRDRFAC, SUITFAC]
facmat = np.stack(facmat_list, axis=2)
SLPROB[:, :, TSTART] = np.mean(facmat[:, :, range(0, 5)], 2)
SLPROB[:, :, TSTART + 1] = SLPROB[:, :, TSTART]
INTRDPROB[:, TSTART + 1] = slprob_0 * np.ones(nnodes)  # dynamic probability of interdiction at nodes

# Initialize Node agents
STOCK[:, TSTART] = NodeTable['Stock']
TOTCPTL[:, TSTART] = NodeTable['Capital']
PRICE[:, TSTART + 1] = PRICE[:, TSTART]
slcpcty_0 = sl_min[0, erun]
slcpcty[0, TSTART + 1] = slcpcty_0

# subjective risk perception with time distortion
twght = timewght_0 * np.ones((nnodes, 1))

# Set-up trafficking netowrk benefit-cost logic  ############
ltcoeff = locthink[0, erun] * np.ones((nnodes, 1))
margval = np.zeros((nnodes, nnodes, TMAX))
for q in range(0, nnodes):
    if len(np.where(ADJ[q, :] == 1)[0]) > 0:
        margval[q, range(q, nnodes), TSTART] = PRICE[range(q, nnodes), TSTART] - PRICE[q, TSTART]

for nd in range(0, ndto):
    idto = np.where(NodeTable['DTO'] == nd + 1)[0]
    margvalset = [idto[x] for x in range(len(idto)) if idto[x] != endnodeset]
    routepref[0, idto, TSTART + 1] = margval[0, idto, 0] / np.amax(margval[0, margvalset])

routepref[:, endnodeset, TSTART + 1] = 1
totslrisk[0, TSTART + 1] = 1

CTRANS[:, :, TSTART + 1] = CTRANS[:, :, TSTART]

# Set-up figure for trafficking movie
MOV = np.zeros((nnodes, nnodes, TMAX))

# Output tables for flows(t) and interdiction prob(t-1)
t = TSTART + 1

FLOW = scipy.io.loadmat(r'C:\Users\htanveer\Narc\NarcoABM_python\data\init_flow_ext.mat')['FLOW']

init = np.where(FLOW[:, :, 0] > 0)
rinit = init[0]
cinit = init[1]

for w in range(0, len(rinit)):
    MOV[rinit[w], cinit[w], 1] = FLOW[rinit[w], cinit[w], 1]

Tflow, Tintrd = intrd_tables_batch(FLOW, slsuccess, SLPROB, NodeTable, EdgeTable, t, erun, mrun)
for m in mr:
    for time in times:
        print("m = ", m, ", time = ", time)
        MTMCI_func(data_sourcing(), time, m)

        intrdct_events, intrdct_nodes = optimize_interdiction_batch(ADJ)
        slevent[:, :, time] = intrdct_events
        slnodes[time].append(intrdct_nodes)
        MOV[:, 0, time] = NodeTable['Stock']

        # Iterate through trafficking nodes
        for n in range(0, nnodes):
            if len(np.where(ADJ[n, :] == 1)[0]) == 0 or n == endnodeset:
                continue

            # Route cocaine shipments #
            STOCK[n, time] = STOCK[n, time - 1] + STOCK[n, time]
            rtdto = NodeTable.loc[np.where(ADJ[n, :] == 1)[0], 'DTO']
            if len(np.where(rtdto == 0)[0]) > 0:
                rtdto[(np.where[rtdto] == 0)[0]] = NodeTable.loc[n, 'DTO']
            CPCTY[n, np.where(ADJ[n, :] == 1)[0]] = basecap[0, erun] * rtcap[
                rtdto - 1, int(np.floor(time / 12)) + 1]
            TOTCPTL[n, time] = TOTCPTL[n, time - 1] + TOTCPTL[n, time]
            if STOCK[n, time] > 0:
                inei = []
                if n > 0:
                    LEAK[n, time] = nodeloss * STOCK[n, time]
                    STOCK[n, time] = STOCK[n, time] - LEAK[n, time]
                elif n == 0:
                    inei = np.intersect1d(np.where(ADJ[n, :] == 1)[0], np.where(routepref[n, :, time] > 0)[0])
                    for nd in range(0, len(np.unique(NodeTable.loc[1:nnodes, 'DTO']))):
                        if len(np.where(NodeTable.loc[inei, 'DTO'] == nd)) == 0:
                            idtombr = np.where(NodeTable['DTO'] == nd)[0]
                            subinei = np.intersect1d(np.where(ADJ[n, idtombr] == 1)[0],
                                                     np.where(routepref[n, idtombr, time] > 0)[0])
                            if len(subinei) == 0:
                                subinei = np.intersect1d(np.where(ADJ[n, idtombr] == 1)[0],
                                                         np.where(routepref[n, idtombr, time] ==
                                                                  np.max(routepref[n, idtombr, time]))[0])
                            np.append(inei, subinei)
                else:
                    inei = np.intersect1d(np.where(ADJ[n, :] == 1)[0], np.where(routepref[n, :, time] > 0)[0])
                    inei = inei[np.isin(inei, np.append(np.where(NodeTable['DTO'] == NodeTable.loc[n, 'DTO'])[0],
                                                        endnodeset))]
                    if len(np.where(inei != 0)[0]) == 0:
                        inei = np.intersect1d(np.where(ADJ[n, :] == 1)[0],
                                              np.where(routepref[n, :, time] == np.max(routepref[n, :, time]))[0])
                        inei = inei[np.isin(inei, np.append(np.where(NodeTable['DTO'] == NodeTable.loc[n, 'DTO'])[0]
                                                            , endnodeset))]


                # Procedure for selecting routes based on expected profit #
                c_trans = CTRANS[n, inei, time].reshape(1, -1)
                p_sl = SLRISK[n, inei].reshape(1, -1)
                y_node = (PRICE[inei, time] - PRICE[n, time]).reshape(-1, 1)
                q_node = np.minimum(STOCK[n, time] / len(inei), CPCTY[n, inei]).reshape(1, -1)
                lccf = ltcoeff[n, 0]
                totstock = STOCK[n, time]
                totcpcty = CPCTY[n, inei].reshape(1, -1)
                rtpref = routepref[n, inei, time].reshape(1, -1)
                dtonei = NodeTable.loc[inei, 'DTO'].to_numpy().reshape(-1, 1)
                """ check NodeTable DTO should have 0 and 1 instead of 1 and 2 - if so remove -1 from line below """
                cutflag = dtocutflag[np.unique(dtonei[np.where(dtonei != 0)]) - 1].reshape(-1, 1)

                # !! If conditions are needed to handle empty values
                neipick, neivalue, valuex = calc_neival(c_trans, p_sl, y_node, q_node, lccf, rtpref, dtonei,
                                                        cutflag, totcpcty, totstock, edgechange)

                neipick = np.int_(neipick)
                # With top-down route optimization
                inei = inei[neipick]
                # weight according to salience value function
                if len(np.where(valuex <= 0)) > 0:
                    WGHT[n, inei] = (1 - SLRISK[n, inei]) / np.sum(1 - SLRISK[n, inei])
                else:
                    WGHT[n, inei] = np.transpose(
                        np.maximum(valuex[neipick], 0) / np.sum(np.amax(valuex[neipick], 0)))

                activeroute[n, time] = inei
                FLOW[n, inei, time] = np.minimum(np.multiply(WGHT[n, inei] / np.sum(WGHT[n, inei]), STOCK[n, time]),
                                                 CPCTY[n, inei])

                OUTFLOW[n, time] = np.sum(FLOW[n, inei, time])
                STOCK[n, time] = STOCK[n, time] - OUTFLOW[n, time]
                nodecosts = np.sum(np.multiply(FLOW[n, inei, time], CTRANS[n, inei, time]))

                # Check for S#L event
                if np.any(np.isin(np.where(slevent[n, :, time] != 0)[0], inei)):
                    isl = np.where(slevent[n, inei, time] == 1)
                    intrdctobs[n, inei[isl], time] = 1
                    intcpt = np.minimum(p_sucintcpt[erun] * NodeTable.loc[inei[isl], 'pintcpt'], 1)

                    # interception probability
                    p_int = np.random.rand(len(intcpt), 1)
                    for p in range(len(intcpt)):
                        if p_int[p] <= intcpt[p]:
                            slsuccess[n, inei[isl[p]], time] = FLOW[n, inei[isl[p]], time]
                            slvalue[n, inei[isl[p]], time] = np.multiply(FLOW[n, inei[isl[p]], time],
                                                                         np.transpose(PRICE[inei[isl[p]], time]))
                            FLOW[n, inei[isl[p]], time] = 0
                        else:
                            slsuccess[n, inei[isl[p]], time] = 0
                            slvalue[n, inei[isl[p]], time] = 0

                    STOCK[inei, time] = STOCK[inei, time] + np.transpose(FLOW[n, inei, time])
                    noderevenue = np.sum(np.multiply(FLOW[n, inei, time], np.transpose(PRICE[inei, time])))
                    TOTCPTL[inei, time] = TOTCPTL[inei, time] - (np.multiply(np.transpose(FLOW[n, inei, time]),
                                                                             PRICE[inei, time]))
                    ICPTL[n, time] = rentcap * np.sum(np.multiply(FLOW[n, inei], ADDVAL[n, inei].reshape(-1, 1))) # need to reshape!!!

                    MARGIN[n, time] = noderevenue - nodecosts + min(TOTCPTL[n, time], 0)
                    if n > 1:
                        BRIBE[n, time] = max(bribepct * MARGIN[n, time], 0)
                        if MARGIN[n, time] > 0:
                            RENTCAP[n, time] = MARGIN[n, time] - BRIBE[n, time]
                        else:
                            RENTCAP[n, time] = MARGIN[n, time]
                        TOTCPTL[n, time] = max(TOTCPTL[n, time], 0) + RENTCAP[n, time]
                    else:
                        RENTCAP[n, time] = MARGIN[n, time]
                        TOTCPTL[n, time] = TOTCPTL[n, time] + RENTCAP[n, time]
                else:
                    STOCK[inei, time] = STOCK[inei, time] + np.transpose(FLOW[n, inei, time])
                    nodecosts = np.sum(np.multiply(FLOW[n, inei, time], CTRANS[n, inei, time]))
                    noderevenue = np.sum(np.multiply(FLOW[n, inei, time], np.transpose(PRICE[inei, time])))
                    TOTCPTL[inei, time] = TOTCPTL[inei, time] - np.multiply(np.transpose(FLOW[n, inei, time]),
                                                                            PRICE[inei, time])
                    ICPTL[n, time] = rentcap * np.sum(np.multiply(FLOW[n, inei], ADDVAL[n, inei].reshape(-1, 1))) # need to reshape/transpose!!!
                    MARGIN[n, time] = noderevenue - nodecosts + min(TOTCPTL[n, time], 0)
                    if n > 1:
                        BRIBE[n, time] = max(bribepct * MARGIN[n, time], 0)
                        if MARGIN[n, time] > 0:
                            RENTCAP[n, time] = MARGIN[n, time] - BRIBE[n, time]
                        else:
                            RENTCAP[n, time] = MARGIN[n, time]
                        TOTCPTL[n, time] = max(TOTCPTL[n, time], 0) + RENTCAP[n, time]
                    else:
                        RENTCAP[n, time] = MARGIN[n, time]
                        TOTCPTL[n, time] = TOTCPTL[n, time] + RENTCAP[n, time]

                # Update perceived risk in response to S&L and Interdiction events
                timeweight = twght[n]

                # identify neighbors in network (without network toolbox)
                fwdnei = inei
                t_eff = np.arange(0, 12)
                if t == TSTART + 1:
                    # Risk perception only updated when successful interdiction takes place
                    sloccur = np.array([[np.zeros((12, len(fwdnei)))], [(slsuccess[n, fwdnei, TSTART + 1] > 0)]])
                elif t > TSTART + 1 and len(fwdnei) == 1:
                    sloccur = np.array([[np.zeros((13 - len(np.arange(np.amax(TSTART + 1, time - 12), time + 1)),
                                                   1))],
                                        [np.squeeze(slsuccess[n, fwdnei, np.arange(np.amax(TSTART + 1, time - 12),
                                                                                   time + 1)] > 0)]])
                else:
                    sloccur = np.array([[np.zeros((13 - len(np.arange(np.amax(TSTART + 1, time - 12), time + 1)),
                                                   len(fwdnei)))],
                                        [np.transpose(np.squeeze(slsuccess[n, fwdnei,
                                        np.arange(np.amax(TSTART + 1, time - 12), time + 1)] > 0))]])

                sl_risk, slevnt, tmevnt = calc_intrisk(sloccur, t_eff, alpharisk, betarisk, timeweight)         # SLRISK[n, fwdnei] = sl_risk # sl_risk:shape(1, 12) --> put into SLRISH[n, fwdnei] = (2,)
                SLRISK[n, fwdnei] = np.sum(np.sum(sl_risk[0]), axis = 0)
                # SLRISK[n, fwdnei] = np.mean(np.mean(sl_risk[0]), axis = 0)

                if len(np.where(sl_risk != 0)) > 0:
                    avgslrisk[n, time] = SLRISK[n, activeroute[n, time]]

                NodeTable['Stock'] = STOCK[:, time]
                NodeTable['Capital'] = TOTCPTL[:, time]
                RISKPREM[:, :, time] = np.amax(np.multiply((1 - delta_rt), RISKPREM[:, :, time - 1]) + np.multiply(
                    delta_rt, ((SLRISK / baserisk[0][erun]) ** riskmltplr[0][erun])), 1) # both baserick & riskmltplr are in [1, 11] shape

                # Make trafficking move
                MOV[:, n, time] = STOCK[:, time]

        # Risk premium on cost of doing business (transport costs)
        CTRANS[:, :, time + 1] = np.multiply(CTRANS[:, :, time], RISKPREM[:, :, time])
        totslrisk[time + 1] = np.mean(np.concatenate(2, avgslrisk[:, time]))

        # Calculate updated marginal profit
        for q in range(0, nnodes):
            if len(np.where(ADJ[q, :] == 1)) == 0:
                margval[q, q + 1: nnodes, time] = PRICE[q + 1: nnodes, time] - PRICE[q, time]

        # Route Optimization ###########
        for dt in range(0, ndto):
            idto = np.where(NodeTable['DTO'] == dt)[0]
            DTOBDGT[dt, time] = np.sum(np.multiply(STOCK[endnodeset, time], PRICE[endnodeset, time]))  # total DTO
            # funds for expansion/viability
            dtorefvec = np.array([[1], [idto], [mexnode]])
            subroutepref = routepref[dtorefvec, dtorefvec, time]
            subflow = FLOW[dtorefvec, dtorefvec, time]
            dtoslsuc = slsuccess[dtorefvec, dtorefvec, time]
            allflows = subflow + dtoslsuc
            # locate active edges
            indices = np.where(allflows > 0)
            irow = indices[0]
            dtoEdgeTable = EdgeTable.loc[:, 'sendedge']
            dtoEdgeTable = dtoEdgeTable[np.isin(dtoEdgeTable['EndNodes'].str[1], dtorefvec), :]
            dtoSLRISK = SLRISK[dtorefvec, dtorefvec]
            dtoADDVAL = margval[dtorefvec, dtorefvec, time]
            dtoCTRANS = CTRANS[dtorefvec, dtorefvec, time]

            # calculate losses from S&L events
            # volume-based - does not matter where in supply chain
            ipossl = np.where(dtoslsuc > 0)[0]
            nrow = ipossl[0]
            ncol = ipossl[1]
            flowvalues = np.multiply(allflows[allflows > 0], (
                    (PRICE[dtorefvec[ncol], time] - PRICE[dtorefvec[irow], time]) - dtoCTRANS[allflows > 0]))
            supplyfit = np.sum(np.multiply(dtoslsuc[ipossl], (
                    (PRICE[dtorefvec[ncol], time] - PRICE[dtorefvec[nrow], time]) - dtoCTRANS[ipossl])))
            losstolval = losstol * np.maximum(flowvalues)
            if len(np.where(supplyfit != 0)[0]) == 0 and len(np.where(losstolval != 0)[0]) == 0:
                supplyfit = 0.1

            # Route capacity constrains flow volumes, need to expand routes
            idtonet = np.delete(dtorefvec, np.in1d(dtorefvec, endnodeset))

            if np.sum(STOCK[idtonet, time]) >= np.amax(dtoEdgeTable['Capacity']):
                supplyfit = max(supplyfit, losstolval * np.sum(STOCK[idtonet, time]) / rtcap[erun])

            # call top-down route optimization
            expmax = expandmax[erun]

            newroutepref, newedgechange = optimizeroute_multidto(dtorefvec, subflow, supplyfit, expmax,
                                                                 subroutepref, dtoEdgeTable, dtoSLRISK, dtoADDVAL,
                                                                 dtoCTRANS, losstolval, dtoslsuc)

            edgechange[dt] = newedgechange

            # Bottom-up route optimization
            routepref[dtorefvec, dtorefvec, time + 1] = newroutepref

        PRICE[:, time + 1] = PRICE[:, time]

        if growthmdl[erun] == 1:
            STOCK[0, time] = stock_0 + (prodgrow[erun] * np.ceil((time - TSTART) / 12))  # additional production to
            # enter network next time step
        elif growthmdl[erun] == 2:
            STOCK[0, time] = (stock_max * stock_0 * np.exp(prodgrow[erun] * int(np.floor(time / 12)))) / (
                    stock_max + stock_0 * (np.exp(prodgrow[erun] * int(np.floor(time / 12))) - 1))

        STOCK[endnodeset, time + 1] = 0
        NodeTable.loc[1, 'Stock'] = STOCK[1, time + 1]
        NodeTable[endnodeset, 'Stock'] = 0
        slcount_edges[time] = len(np.where(slsuccess[:, :, time] > 0)[0])
        h_slsuccess = slsuccess[:, :, time]
        slcount_vol[time] = np.sum(h_slsuccess[h_slsuccess > 0])

        # Output tables for flows(t) and interdiction prob(t-1)
        Tflow, Tintrd = intrd_tables_batch(FLOW, slsuccess, SLPROB, NodeTable, EdgeTable, t, erun, m)
        data_processing(Tflow, time, m)

m =  0 , time =  1
Objective Value: 25544.195141046
MTMCLP for timestep 1, model run 0 is complete.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
neipick, neivalue, valuex = calc_neival(c_trans, p_sl, y_node, q_node, lccf, rtpref, dtonei,
                                                        cutflag, totcpcty, totstock, edgechange)

IndexError: index 0 is out of bounds for axis 0 with size 0